In [1]:
import os
from dataclasses import dataclass
import logging
logging.basicConfig(level=logging.INFO, 
    format='%(asctime)s %(levelname)s - %(message)s',
    datefmt="%H:%M:%S"
)

import ioh # version 0.3.2.8.3
import numpy as np
%rm -rf my-experiment*

# Problem
In ioh, everything revolves around the `problem` class, which exists for both `Real` and `Integer` types for continious and discrete problems resp. These classes are wrappers around an objective function, and can be used to interact with the various other parts of iohexperimenter. 

We have a number of objective functions already implemented for convenience, which includes the objective functions from the BBOB single objective benchmark by the COCO platform for the continous case and the PBO benchmark functions for the discrete case.


In [2]:
## A list of problems can be accessed via the base classes
ioh.problem.Real.problems

{1: 'Sphere',
 2: 'Ellipsoid',
 3: 'Rastrigin',
 4: 'BuecheRastrigin',
 5: 'LinearSlope',
 6: 'AttractiveSector',
 7: 'StepEllipsoid',
 8: 'Rosenbrock',
 9: 'RosenbrockRotated',
 10: 'EllipsoidRotated',
 11: 'Discus',
 12: 'BentCigar',
 13: 'SharpRidge',
 14: 'DifferentPowers',
 15: 'RastriginRotated',
 16: 'Weierstrass',
 17: 'Schaffers10',
 18: 'Schaffers1000',
 19: 'GriewankRosenBrock',
 20: 'Schwefel',
 21: 'Gallagher101',
 22: 'Gallagher21',
 23: 'Katsuura',
 24: 'LunacekBiRastrigin'}

In [3]:
# In order to instatiate a problem instance, we can do the following:
problem = ioh.get_problem(
    "Sphere", 
    instance=1,
    dimension=10,
    problem_type="Real"
)
problem

<RealProblem 1. Sphere (iid=1 dim=10)>

In [4]:
# The problem class includes information about the problem, which can be retrieved via the meta_data accessor
problem.meta_data

<MetaData: Sphere id: 1 iid: 1 dim: 10>

In [5]:
# The current state of the problem, e.g. the number of evaluations, best seen points etc. are stored in the problems state.
problem.state

<State evaluations: 0 optimum_found: false current_best: <Solution x: {nan, nan, nan, nan, nan, nan, nan, nan, nan, nan} y: nan>>

In [6]:
# We can access the contraint information of the problem
x0 = np.random.uniform(problem.constraint.lb, problem.constraint.ub)

# Evaluation happens like a 'normal' objective function would
problem(x0)

# Whenever the problem is evaluated, the state changes
problem.state

<State evaluations: 1 optimum_found: false current_best: <Solution x: {-1.2950204717511795, 0.327119000894486, 0.615587210650582, -0.8015767595153047, 1.9372506289377602, -3.0763003666405453, 4.170721104709955, -1.4149184991965824, 0.8921392997525395, -4.286495245334493} y: 155.66359663903182>>

In [7]:
@dataclass
class RandomSearch:
    'Simple random search algorithm'
    
    n: int
    length: float = 0.0
        
    def __call__(self, problem: ioh.problem.Real) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        
        for _ in range(self.n):
            # We can use the problems constraint accessor to get information about the problem bounds
            x = np.random.uniform(problem.constraint.lb, problem.constraint.ub)
            self.length = np.linalg.norm(x)
            
            problem(x)         

In [8]:
# If we want to perform multiple runs with the same objective function, after every run, the problem has to be reset, 
# such that the internal state reflects the current run.

def run_experiment(problem, algorithm, n_runs=5):
    for run in range(n_runs):
        
        # Run the algorithm on the problem
        algorithm(problem)

        # print the best found for this run
        print(f"run: {run+1} - best found:{problem.state.current_best.y: .3f}")

        # Reset the problem
        problem.reset()

run_experiment(problem, RandomSearch(10))

run: 1 - best found: 125.712
run: 2 - best found: 104.112
run: 3 - best found: 159.729
run: 4 - best found: 135.021
run: 5 - best found: 137.280


# Adding custom problems
As the list of already implemented problems might not contains the objective function you would like to analyze and benchmark, we include an easy to use interface to wrap your benchmark function into an iohexperimenter `problem` object. 

Currently, only single objective functions are supported, so the only requirement on the function is its signature, which should take a single array parameter, and return a single floating point number $f(\mathbf{x}) \mapsto \mathbb{R}$.

In this example we will add the function (Styblinski–Tang):

$f(\mathbf{x}) = \frac{\sum_{i=1}^n x_i^4 - 16x_i^2 + 5x_i}{2}$

defined on $[-5, 5]$

In [9]:
def styblinski_tang(x: np.ndarray) -> float:
    return np.sum(np.power(x, 4) - (16 * np.power(x, 2)) + (5 * x)) / 2


styblinski_tang(np.array([-2.903534]*10)) # global minima

-391.661657037714

In [10]:
# we can wrap this function in ioh, as this is is a continous function, we use wrap_real_problem:
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     # Handle to the function
    name="StyblinskiTang",                               # Name to be used when instantiating
    optimization_type=ioh.OptimizationType.Minimization, # Specify that we want to minimize
    lb=-5,                                               # The lower bound
    ub=5,                                                # The upper bound
)

In [11]:
# We can create an instance of this problem, wrapped in ioh
problem = ioh.get_problem("StyblinskiTang", dimension=10)
problem

<RealProblem 25. StyblinskiTang (iid=1 dim=10)>

## Specifying instances
When doing bechmarking of custom problems, it can often be usefull to look at different instances of the same problem, but that perform some transformation the parameter or objective space, to test if an algorithm is invariant to such transformations. 

Here we add the following transformations to the aforementioned objective function:

$T_x(\mathbf{x}, c) = \mathbf{x} + c$

$T_y(y, c) = y * c$

We can do this by providing transformation functions to the `wrap_problem` interface. Note that these take two parameters:
 1. The operand they function on, which are the variables for the variables transformation and objective function value for the objective transformation. 
 2. An integer identifier of the instance id which is currently used. This allows you to specify alternate behavior for different instances.  

In [12]:
# Variables transformation R^d -> R^d
def transform_variables(x: np.ndarray, instance_id:int) -> np.ndarray:
    c = (instance_id - 1) * .5
    return x + c

# Objective transformation R -> R
def transform_objectives(y: float, instance_id:int) -> float:
    c = instance_id
    return y * c

# Note that we can overwrite a previously defined problem by calling wrap_real_problem again with the same name
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     
    name="StyblinskiTang",                               
    optimization_type=ioh.OptimizationType.Minimization, 
    lb=-5,                                               
    ub=5,      
    
    # Adding the transformation functions
    transform_variables=transform_variables,     
    transform_objectives=transform_objectives
)

In [13]:
# We can now create different instances of the same problem
instance1 = ioh.get_problem("StyblinskiTang", instance=1, dimension=10)
instance2 = ioh.get_problem("StyblinskiTang", instance=2, dimension=10)
instance1, instance2

(<RealProblem 25. StyblinskiTang (iid=1 dim=10)>,
 <RealProblem 25. StyblinskiTang (iid=2 dim=10)>)

In [14]:
# Note that when evaluating with the same point, each instance gives a different (transformed value)
instance1(x0), instance2(x0)

(-68.11304809829876, -88.46879515525143)

# Logging data
The default usage of IOHExperimenter is in generating logs of benchmarking experiments which can be analyzed in IOHAnalyzer. 

In [15]:
import os

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="my-experiment",       # in a folder named: 'my-experiment'
    algorithm_name="random-search",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

# this automatically creates a folder 'my-experiment' in the current working directory
# if the folder already exists, it will given an additional number to make the name unique
logger

<Analyzer /home/jacob/code/ioh-tutorial/my-experiment>

In [16]:
%ls

'IOHExperimenter Tutorial.ipynb'   my-experiment/   requirements.txt   venv/


In [17]:
# In order to log data for a problem, we only have to attach it to a logger
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)

# We can then run the random search as before, only now all data will be logged to a file
run_experiment(problem, RandomSearch(10), n_runs=1)

run: 1 - best found:-60.648


In [18]:
%cat my-experiment/IOHprofiler_f25_StyblinskiTang.info

suite = "unknown_suite", funcId = 25, funcName = "StyblinskiTang", DIM = 2, maximization = "F", algId = "random-search", algInfo = "algorithm_info"
%
data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat, 1:10|-60.6482

In [19]:
%cat my-experiment/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

"function evaluation" "current f(x)" "best-so-far f(x)" "current af(x)+b" "best af(x)+b" x0 x1
1 87.0248990684 87.0248990684 87.0248990684 87.0248990684 -2.016090 4.948675
2 45.8853303104 45.8853303104 45.8853303104 45.8853303104 4.436211 0.614896
5 -15.3107416022 -15.3107416022 -15.3107416022 -15.3107416022 0.399926 1.723550
8 -58.0688922641 -58.0688922641 -58.0688922641 -58.0688922641 -3.823673 -2.688763
10 -60.6482247986 -60.6482247986 -60.6482247986 -60.6482247986 2.963955 -3.281878


## Triggers 
The default behavior of the `Analyzer` logger is to log data only when there is an improvement of the objective value. We can change this behaviour, by specifying one or more triggers, which are logical operators, which when one of them evaluates to True, will cause data to be logged.

We provide a number of trigger variants which can be use to customize the logging. In the following example, a trigger is defined which evaluates to True, every 3 function evaluations. It is combined with a trigger for improvement, so data will be logged on every 3rd function evaluation, or when there is an observed improvement of the objective value

In [20]:
triggers = [
    ioh.logger.trigger.Each(3),
    ioh.logger.trigger.OnImprovement()
]

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True,
    
    # Add the triggers to the logger
    triggers = triggers
)

logger

<Analyzer /home/jacob/code/ioh-tutorial/my-experiment-1>

In [21]:
# Rerun the same experiment as before
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)
run_experiment(problem, RandomSearch(10), n_runs=1)

run: 1 - best found:-48.744


In [22]:
# We can now see that data is logged either if there is improvement, or on every 3rd evaluation
%cat my-experiment-1/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

"function evaluation" "current f(x)" "best-so-far f(x)" "current af(x)+b" "best af(x)+b" x0 x1
1 -9.4665831884 -9.4665831884 -9.4665831884 -9.4665831884 0.363030 -0.958781
2 -41.6085866610 -41.6085866610 -41.6085866610 -41.6085866610 -2.550476 -0.602979
3 66.9618178073 -41.6085866610 66.9618178073 -41.6085866610 -4.209536 -4.750636
5 -48.7435849457 -48.7435849457 -48.7435849457 -48.7435849457 2.113011 -1.961119
6 -43.4350363835 -48.7435849457 -43.4350363835 -48.7435849457 2.524879 -1.492339
9 33.8165932089 -48.7435849457 33.8165932089 -48.7435849457 0.981025 -4.561419
10 -27.0036459196 -48.7435849457 -27.0036459196 -48.7435849457 -0.678437 3.185457


## Properties
If we want to keep track of any dynamic parameters a given algorithm might have, we can use properties to log them to the output files. 

In the following example, we will track the length parameters for the RandomSearch algorithm, which is added for illustrative purpoposes, and changes for every function evaluation

In [23]:
# RandomSearch has a length parameter, which is dynamic
algorithm = RandomSearch(10)

# Creating a new logger
logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True
)

# Before we attach a problem, we tell the logger to keep track of the length parameter on algorithm
logger.watch(algorithm, "length")

# We can now again run the same experiment 
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)

problem.attach_logger(logger)
run_experiment(problem, algorithm, n_runs=1)

run: 1 - best found:-54.528


In [24]:
# Note the additional length parameter being logged 
%cat my-experiment-2/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

"function evaluation" "current f(x)" "best-so-far f(x)" "current af(x)+b" "best af(x)+b" length x0 x1
1 -5.3173094017 -5.3173094017 -5.3173094017 -5.3173094017 4.0721437769 -4.069792 0.138389
3 -36.4236218156 -36.4236218156 -36.4236218156 -36.4236218156 3.1209839686 2.748852 1.477956
8 -49.9689041343 -49.9689041343 -49.9689041343 -49.9689041343 4.8216253602 -3.312845 3.503302
9 -54.5278780884 -54.5278780884 -54.5278780884 -54.5278780884 3.1210723573 -2.337390 2.068261
10 61.1737368870 -54.5278780884 61.1737368870 -54.5278780884 5.0319882776 4.690094 1.823163


# Alternate logging behaviour
We provide a number of different loggers in addition to the `Analyzer` logger, which include:
 - `FlatFile` which logs data to a simple csv file
 - `Store` which keeps all of the stored data in memory
 - `EAF/EAH` which compute Empirical Attainment Function/Histogram statistics on the fly

You can define your own custom logging behavoir by inheriting from the `AbstractLogger` class. The only required part of the interface is that you override the `__call__` operator, which takes a single `ioh.LogInfo` parameter. In this method you should define your desired behavior. 

In [25]:
# Simple logger that logs data using the python logging module whenever it is triggeredd
class MyLogger(ioh.logger.AbstractLogger):
    def __call__(self, log_info: ioh.LogInfo):
        logging.info(msg=f"triggered! y: {log_info.current.y}")


# The abstract logger takes two parameters, triggers and properties
mylogger = MyLogger(triggers=[ioh.logger.trigger.ON_IMPROVEMENT])

problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(mylogger)

run_experiment(problem, RandomSearch(10), 1)

14:43:59 INFO - triggered! y: 8.159473731241318
14:43:59 INFO - triggered! y: -37.76967367460152
14:43:59 INFO - triggered! y: -56.775703457009584


run: 1 - best found:-56.776


In [26]:
# Creating a logger which tracks the current best search point on improvement in memory
store = ioh.logger.Store(
    triggers=[ioh.logger.trigger.ON_IMPROVEMENT], 
    properties=[ioh.logger.property.CURRENT_Y_BEST]
)

# Create a MyLogger which triggers at every evaluation
mylogger = MyLogger(triggers=[ioh.logger.trigger.ALWAYS])

# It is possible to combine the behaviour of multiple loggers, using the combine wrapper
combined_logger = ioh.logger.Combine([store, mylogger])

In [27]:
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(combined_logger)

run_experiment(problem, RandomSearch(10), 1)

14:43:59 INFO - triggered! y: -44.966608962809694
14:43:59 INFO - triggered! y: -40.06694479879753
14:43:59 INFO - triggered! y: -23.645735366568896
14:43:59 INFO - triggered! y: -67.67776890562396
14:43:59 INFO - triggered! y: -37.87146940319561
14:43:59 INFO - triggered! y: -44.32822127144331
14:43:59 INFO - triggered! y: -60.711008231986185
14:43:59 INFO - triggered! y: -11.894532038951056
14:43:59 INFO - triggered! y: -17.361515573428903
14:43:59 INFO - triggered! y: -78.28618270355574


run: 1 - best found:-78.286


In [28]:
store.data()

{'None': {25: {2: {1: {0: {0: {'current_y': -44.966608962809694},
      1: {'current_y': -67.67776890562396},
      2: {'current_y': -78.28618270355574}}}}}}}